In [ ]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
import joblib
import os
import warnings
warnings.filterwarnings('ignore')

output_dir = 'notebooks/XGBoost'

def standardize_region_names(df):
    """Стандартизация названий регионов"""
    df_clean = df.copy()
    
    replacements = {
        'Ненецкий авт.округ': 'Ненецкий автономный округ',
        'Hенецкий авт.округ': 'Ненецкий автономный округ',
        '  Ненецкий автономный округ': 'Ненецкий автономный округ',
        
        'Ямало-Ненецкий авт.округ': 'Ямало-Ненецкий автономный округ',
        'Ямало-Hенецкий авт.округ': 'Ямало-Ненецкий автономный округ',
        '  Ямало-Ненецкий автономный округ': 'Ямало-Ненецкий автономный округ',
        
        'Ханты-Мансийский авт.округ-Югра': 'Ханты-Мансийский автономный округ - Югра',
        '  Ханты-Мансийский автономный округ - Югра': 'Ханты-Мансийский автономный округ - Югра',
        
        'Республика Татарстан(Татарстан)': 'Республика Татарстан',
        'Чувашская Республика(Чувашия)': 'Чувашская Республика',
        'Республика Северная Осетия- Алания': 'Республика Северная Осетия-Алания',
        
        'Oмская область': 'Омская область',
        'Hижегородская область': 'Нижегородская область',
        
        'г. Севастополь': 'г.Севастополь',
        'г.Москва': 'г.Москва',
        'г.Санкт-Петербург': 'г.Санкт-Петербург',
        
        'Чукотский авт.округ': 'Чукотский автономный округ',
        'Чукотский автономный округ': 'Чукотский автономный округ'
    }
    
    df_clean['Регион'] = df_clean['Регион'].replace(replacements)
    df_clean['Регион'] = df_clean['Регион'].str.strip()
    
    return df_clean

class OPJXGBoostForecasterAdvanced:
    def __init__(self, random_state=42):
        self.model = None
        self.scaler = StandardScaler()
        self.feature_names = []
        self.is_fitted = False
        self.last_known_data = None
        self.random_state = random_state
        self.first_year = None
        self.validation_results = None
        
    def clean_numeric_columns(self, df):
        """
        Очистка числовых колонок
        """
        df = df.copy()
        
        numeric_columns = [col for col in df.columns if col not in ['Регион', 'Год', 'ОПЖ']]
        
        for col in numeric_columns:
            if df[col].dtype == 'object':
                df[col] = (df[col]
                          .astype(str)
                          .str.replace('\xa0', '')
                          .str.replace(' ', '')
                          .str.replace(',', '.')
                          .str.replace('−', '-')
                          .str.replace('–', '-')
                          )
                df[col] = pd.to_numeric(df[col], errors='coerce')
        
        if 'ОПЖ' in df.columns:
            df['ОПЖ'] = pd.to_numeric(df['ОПЖ'], errors='coerce')
        
        return df
        
    def prepare_features(self, df, is_training=True):
        """
        Подготовка признаков для ОПЖ с сохранением данных с 2014 года
        """
        df = df.copy().sort_values(['Регион', 'Год'])
        
        if self.first_year is None:
            self.first_year = df['Год'].min()
        
        print(f"Исходные данные: {df['Год'].min()}-{df['Год'].max()}, {len(df)} строк")
        
        # Создание лаговых признаков
        if is_training or df['Год'].min() < 2024:
            df['lag1_ОПЖ'] = df.groupby('Регион')['ОПЖ'].shift(1)
            df['lag2_ОПЖ'] = df.groupby('Регион')['ОПЖ'].shift(2)
            df['lag1_Число умерших'] = df.groupby('Регион')['Число умерших'].shift(1)
            df['lag1_Младенческая смертность коэф'] = df.groupby('Регион')['Младенческая смертность коэф'].shift(1)
            df['lag1_Общая численность инвалидов'] = df.groupby('Регион')['Общая численность инвалидов'].shift(1)
            
            df['ОПЖ_MA2'] = df.groupby('Регион')['ОПЖ'].transform(lambda x: x.rolling(2, min_periods=1).mean())
            df['ОПЖ_MA3'] = df.groupby('Регион')['ОПЖ'].transform(lambda x: x.rolling(3, min_periods=1).mean())
        
        df['year_trend'] = df['Год'] - self.first_year
        df['год_от_начала'] = df['Год'] - self.first_year
        
        # Создание производных медицинских и социальных показателей
        df['Врачей_на_10k'] = df['Численность врачей всех специальностей'] / df['Численность населения'] * 10000
        df['Умерших_на_1000'] = df['Число умерших'] / df['Численность населения'] * 1000
        df['Инвалидов_на_1000'] = df['Общая численность инвалидов'] / df['Численность населения'] * 1000
        df['Преступлений_на_1000'] = df['Кол-во преступлений'] / df['Численность населения'] * 1000
        df['Браков_на_1000'] = df['Браков'] / df['Численность населения'] * 1000
        df['Разводов_на_1000'] = df['Разводов'] / df['Численность населения'] * 1000
        
        df['Индекс_здравоохранения'] = (
            df['Врачей_на_10k'] + 
            df['Число больничных организаций на конец отчетного года'] + 
            df['Число санаторно-курортных организаций']
        ) / 3
        
        df['Социальный_индекс'] = (
            df['Средняя ЗП'] / df['Величина прожиточного минимума'] - 
            (df['Уровень бедности'] / 100)
        )
        
        df['изменение_населения'] = df.groupby('Регион')['Численность населения'].pct_change()
        df['изменение_ВРП'] = df.groupby('Регион')['Валовой региональный продукт на душу населения (ОКВЭД 2)'].pct_change()
        
        if is_training:
            print(f"Данные до обработки NaN: {len(df)} строк")
            
            numeric_cols = df.select_dtypes(include=[np.number]).columns
            for col in numeric_cols:
                if col not in ['Год']:
                    df[col] = df.groupby('Регион')[col].transform(
                        lambda x: x.fillna(x.median()) if not x.isnull().all() else x
                    )
            
            df = df.fillna(df.median(numeric_only=True))
            
            print(f"Данные после обработки NaN: {len(df)} строк")
            print(f"Годы после обработки: {df['Год'].min()}-{df['Год'].max()}")
        
        return df
    
    def train_validate_split(self, df, train_end_year=2021, validation_year=2022, test_year=2023):
        """
        Разделение на обучающую, валидационную и тестовую выборки
        """
        train_mask = df['Год'] <= train_end_year
        val_mask = df['Год'] == validation_year
        test_mask = df['Год'] == test_year
        
        return train_mask, val_mask, test_mask
    
    def fit_with_validation(self, df, train_end_year=2021, validation_year=2022):
        """
        Обучение с валидацией на 2022 году
        """
        print(f"\n{'='*70}")
        print(f"ОБУЧЕНИЕ МОДЕЛИ ОПЖ НА ДАННЫХ ДО {train_end_year} ГОДА")
        print(f"ВАЛИДАЦИЯ НА {validation_year} ГОДУ")
        print(f"{'='*70}")
        
        # Очистка и подготовка данных
        df_clean = self.clean_numeric_columns(df)
        df_processed = self.prepare_features(df_clean, is_training=True)
        
        # Разделение на обучающую и валидационную выборки
        train_mask, val_mask, _ = self.train_validate_split(df_processed, train_end_year, validation_year)
        
        # Определение признаков
        self.feature_names = [
            'Численность населения', 'Число умерших', 'Общая численность инвалидов',
            'Браков', 'Разводов', 'Младенческая смертность коэф', 
            'Численность врачей всех специальностей', 
            'Число больничных организаций на конец отчетного года', 
            'Число санаторно-курортных организаций',
            'Валовой региональный продукт на душу населения (ОКВЭД 2)',
            'Величина прожиточного минимума', 'Уровень бедности', 'Средняя ЗП',
            'Кол-во преступлений',
            'lag1_ОПЖ', 'lag2_ОПЖ', 'lag1_Число умерших', 
            'lag1_Младенческая смертность коэф', 'lag1_Общая численность инвалидов',
            'ОПЖ_MA2', 'ОПЖ_MA3', 'year_trend', 'год_от_начала',
            'Врачей_на_10k', 'Умерших_на_1000', 'Инвалидов_на_1000', 
            'Преступлений_на_1000', 'Браков_на_1000', 'Разводов_на_1000',
            'Индекс_здравоохранения', 'Социальный_индекс',
            'изменение_населения', 'изменение_ВРП'
        ]
        
        self.feature_names = [f for f in self.feature_names if f in df_processed.columns]
        print(f"Используется {len(self.feature_names)} признаков")
        
        # Подготовка данных
        X_train = df_processed[train_mask][self.feature_names]
        X_val = df_processed[val_mask][self.feature_names]
        y_train = df_processed[train_mask]['ОПЖ']
        y_val = df_processed[val_mask]['ОПЖ']
        
        print(f"Обучающая выборка: {X_train.shape} (годы: {df_processed[train_mask]['Год'].min()}-{df_processed[train_mask]['Год'].max()})")
        print(f"Валидационная выборка: {X_val.shape} (год: {validation_year})")
        
        # Масштабирование
        scale_features = [f for f in self.feature_names if not f.startswith(('lag', 'ОПЖ_MA', 'изменение_'))]
        X_train_scaled = X_train.copy()
        X_val_scaled = X_val.copy()
        
        X_train_scaled[scale_features] = self.scaler.fit_transform(X_train[scale_features])
        X_val_scaled[scale_features] = self.scaler.transform(X_val[scale_features])
        
        # Обучение модели с улучшенными параметрами
        self.model = xgb.XGBRegressor(
            max_depth=6,
            learning_rate=0.03,
            n_estimators=400,
            random_state=self.random_state,
            n_jobs=-1,
            subsample=0.7,
            colsample_bytree=0.7,
            reg_alpha=0.1,
            reg_lambda=1.0
        )
        
        print("\nОбучение модели...")
        self.model.fit(X_train_scaled, y_train)
        self.is_fitted = True
        
        # Сохраняем последние известные данные для прогноза
        self.last_known_data = df_processed[df_processed['Год'] == train_end_year].copy()
        
        # Валидация на 2022 году
        y_val_pred = self.model.predict(X_val_scaled)
        
        # Расчет метрик валидации
        self.validation_results = {
            'y_true': y_val,
            'y_pred': y_val_pred,
            'year': validation_year,
            'metrics': self._calculate_detailed_metrics(y_val, y_val_pred, "Валидация")
        }
        
        print(f"\n{'='*70}")
        print("РЕЗУЛЬТАТЫ ВАЛИДАЦИИ:")
        print(f"{'='*70}")
        self.print_metrics(self.validation_results['metrics'])
        
        # Кросс-валидация на обучающих данных
        self.cross_validation(X_train_scaled, y_train)
        
        # Сохранение результатов
        self.results = {
            'X_train': X_train_scaled, 'X_val': X_val_scaled,
            'y_train': y_train, 'y_val': y_val, 'y_val_pred': y_val_pred,
            'df_processed': df_processed,
            'train_years': f"{df_processed[train_mask]['Год'].min()}-{train_end_year}",
            'validation_year': validation_year
        }
        
        return self
    
    def test_on_2023(self, df):
        """
        Тестирование модели на 2023 году (реальные данные)
        """
        print(f"\n{'='*70}")
        print("ТЕСТИРОВАНИЕ МОДЕЛИ НА 2023 ГОДУ (РЕАЛЬНЫЕ ДАННЫЕ)")
        print(f"{'='*70}")
        
        df_clean = self.clean_numeric_columns(df)
        df_processed = self.prepare_features(df_clean, is_training=False)
        
        # Фильтруем данные за 2023 год
        test_mask = df_processed['Год'] == 2023
        
        if test_mask.sum() == 0:
            print("Нет данных за 2023 год для тестирования!")
            return None
        
        X_test = df_processed[test_mask][self.feature_names]
        y_test = df_processed[test_mask]['ОПЖ']
        
        print(f"Тестовая выборка: {X_test.shape} (год: 2023)")
        
        # Масштабирование
        scale_features = [f for f in self.feature_names if not f.startswith(('lag', 'ОПЖ_MA', 'изменение_'))]
        X_test_scaled = X_test.copy()
        X_test_scaled[scale_features] = self.scaler.transform(X_test[scale_features])
        
        # Прогноз
        y_test_pred = self.model.predict(X_test_scaled)
        
        # Расчет метрик
        test_metrics = self._calculate_detailed_metrics(y_test, y_test_pred, "Тестирование на 2023")
        
        print(f"\n{'='*70}")
        print("РЕЗУЛЬТАТЫ ТЕСТИРОВАНИЯ НА 2023 ГОДУ:")
        print(f"{'='*70}")
        self.print_metrics(test_metrics)
        
        # Сравнение с валидацией
        if self.validation_results:
            print(f"\nСРАВНЕНИЕ С ВАЛИДАЦИЕЙ НА {self.validation_results['year']} ГОДУ:")
            print(f"RMSE валидация: {self.validation_results['metrics']['RMSE']:.4f}")
            print(f"RMSE тест 2023: {test_metrics['RMSE']:.4f}")
            print(f"Изменение ошибки: {(test_metrics['RMSE'] - self.validation_results['metrics']['RMSE'])/self.validation_results['metrics']['RMSE']*100:+.2f}%")
        
        # Сохраняем результаты тестирования
        self.test_results = {
            'y_true': y_test,
            'y_pred': y_test_pred,
            'year': 2023,
            'metrics': test_metrics,
            'X_test': X_test_scaled,
            'df_test': df_processed[test_mask].copy()
        }
        
        # Обновляем последние известные данные до 2023 года для точного прогноза
        self.last_known_data = df_processed[df_processed['Год'] == 2023].copy()
        
        return test_metrics
    
    def _calculate_detailed_metrics(self, y_true, y_pred, context=""):
        """Расчет детализированных метрик"""
        mse = mean_squared_error(y_true, y_pred)
        mae = mean_absolute_error(y_true, y_pred)
        rmse = np.sqrt(mse)
        r2 = r2_score(y_true, y_pred)
        
        # Дополнительные метрики
        mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100
        max_error = np.max(np.abs(y_true - y_pred))
        median_error = np.median(np.abs(y_true - y_pred))
        
        # Процентили ошибок
        error_percentiles = np.percentile(np.abs(y_true - y_pred), [25, 50, 75, 90, 95])
        
        return {
            'MSE': mse, 'MAE': mae, 'RMSE': rmse, 'R2': r2,
            'MAPE': mape, 'MaxError': max_error, 'MedianError': median_error,
            'Error_25p': error_percentiles[0], 'Error_50p': error_percentiles[1],
            'Error_75p': error_percentiles[2], 'Error_90p': error_percentiles[3],
            'Error_95p': error_percentiles[4],
            'Context': context
        }
    
    def print_metrics(self, metrics):
        """Красивый вывод метрик"""
        print(f"RMSE: {metrics['RMSE']:.4f} лет")
        print(f"MAE: {metrics['MAE']:.4f} лет")
        print(f"R²: {metrics['R2']:.4f}")
        print(f"MAPE: {metrics['MAPE']:.2f}%")
        print(f"Максимальная ошибка: {metrics['MaxError']:.2f} лет")
        print(f"Медианная ошибка: {metrics['MedianError']:.2f} лет")
        print(f"90% перцентиль ошибок: {metrics['Error_90p']:.2f} лет")
    
    def cross_validation(self, X_train, y_train, cv=5):
        """
        Кросс-валидация для оценки устойчивости модели
        """
        print("\nКросс-валидация на обучающих данных:")
        
        temp_model = xgb.XGBRegressor(
            max_depth=6,
            learning_rate=0.03,
            n_estimators=200,
            random_state=self.random_state,
            n_jobs=-1,
            subsample=0.7,
            colsample_bytree=0.7
        )
        
        try:
            scores = cross_val_score(temp_model, X_train, y_train, 
                                   scoring='neg_mean_squared_error', cv=cv)
            rmse_scores = np.sqrt(-scores)
            print(f"Средний RMSE: {rmse_scores.mean():.4f} (+/- {rmse_scores.std() * 2:.4f})")
            print(f"Диапазон: {rmse_scores.min():.4f} - {rmse_scores.max():.4f}")
        except Exception as e:
            print(f"Ошибка при кросс-валидации: {e}")
    
    def predict_future_advanced(self, future_years=[2024, 2025]):
        """
        Продвинутый прогноз на будущие периоды с использованием лучшей модели
        """
        if not self.is_fitted:
            print("Сначала обучите модель!")
            return None
        
        if self.last_known_data is None:
            print("Нет данных для прогноза!")
            return None
        
        print(f"\n{'='*70}")
        print(f"ПРОГНОЗ ОПЖ НА {future_years} ГОДЫ")
        print(f"{'='*70}")
        
        all_predictions = []
        
        for year_idx, year in enumerate(future_years):
            print(f"\nПрогноз на {year} год:")
            
            year_predictions = []
            current_data = self.last_known_data.copy()
            
            for _, last_row in current_data.iterrows():
                future_row = last_row.copy()
                future_row['Год'] = year
                future_row['year_trend'] = year - self.first_year
                future_row['год_от_начала'] = year - self.first_year
                
                # Более реалистичные коэффициенты роста/снижения
                growth_factors = {
                    'Численность населения': 1.002,  # Скромный рост
                    'Число умерших': 0.992,  # Небольшое снижение
                    'Общая численность инвалидов': 1.0,  # Стабильно
                    'Младенческая смертность коэф': 0.97,  # Снижение
                    'Валовой региональный продукт на душу населения (ОКВЭД 2)': 1.018,
                    'Средняя ЗП': 1.045,
                    'Численность врачей всех специальностей': 1.015,
                    'Величина прожиточного минимума': 1.032,
                    'Уровень бедности': 0.985,
                    'Браков': 1.008,
                    'Разводов': 1.003,
                    'Кол-во преступлений': 0.995,
                    'Число больничных организаций на конец отчетного года': 1.002,
                    'Число санаторно-курортных организаций': 1.001
                }
                
                for col, factor in growth_factors.items():
                    if col in future_row:
                        # Добавляем небольшую случайную вариацию для разных регионов
                        region_factor = 1.0 + np.random.normal(0, 0.01)
                        future_row[col] = last_row[col] * factor * region_factor
                
                # Лаги из предыдущего прогноза
                if year_idx == 0:
                    future_row['lag1_ОПЖ'] = last_row['ОПЖ']
                    future_row['lag2_ОПЖ'] = last_row.get('lag1_ОПЖ', last_row['ОПЖ'])
                else:
                    # Для второго года прогноза используем прогноз первого года
                    prev_year_pred = all_predictions[-len(current_data)]['ОПЖ_прогноз']
                    future_row['lag1_ОПЖ'] = prev_year_pred
                    future_row['lag2_ОПЖ'] = last_row['ОПЖ']
                
                future_row['lag1_Число умерших'] = future_row['Число умерших']
                future_row['lag1_Младенческая смертность коэф'] = future_row['Младенческая смертность коэф']
                future_row['lag1_Общая численность инвалидов'] = future_row['Общая численность инвалидов']
                
                # Скользящие средние
                future_row['ОПЖ_MA2'] = (future_row['lag1_ОПЖ'] + future_row['lag2_ОПЖ']) / 2
                future_row['ОПЖ_MA3'] = (future_row['lag1_ОПЖ'] + future_row['lag2_ОПЖ'] + 
                                         last_row.get('lag2_ОПЖ', future_row['lag2_ОПЖ'])) / 3
                
                # Пересчет производных показателей
                future_row['Врачей_на_10k'] = future_row['Численность врачей всех специальностей'] / future_row['Численность населения'] * 10000
                future_row['Умерших_на_1000'] = future_row['Число умерших'] / future_row['Численность населения'] * 1000
                future_row['Инвалидов_на_1000'] = future_row['Общая численность инвалидов'] / future_row['Численность населения'] * 1000
                future_row['Преступлений_на_1000'] = future_row['Кол-во преступлений'] / future_row['Численность населения'] * 1000
                future_row['Браков_на_1000'] = future_row['Браков'] / future_row['Численность населения'] * 1000
                future_row['Разводов_на_1000'] = future_row['Разводов'] / future_row['Численность населения'] * 1000
                future_row['Индекс_здравоохранения'] = (
                    future_row['Врачей_на_10k'] + 
                    future_row['Число больничных организаций на конец отчетного года'] + 
                    future_row['Число санаторно-курортных организаций']
                ) / 3
                future_row['Социальный_индекс'] = (
                    future_row['Средняя ЗП'] / future_row['Величина прожиточного минимума'] - 
                    (future_row['Уровень бедности'] / 100)
                )
                future_row['изменение_населения'] = (future_row['Численность населения'] - last_row['Численность населения']) / last_row['Численность населения']
                future_row['изменение_ВРП'] = (future_row['Валовой региональный продукт на душу населения (ОКВЭД 2)'] - last_row['Валовой региональный продукт на душу населения (ОКВЭД 2)']) / last_row['Валовой региональный продукт на душу населения (ОКВЭД 2)']
                
                year_predictions.append(future_row)
            
            # Прогноз на конкретный год
            future_df = pd.DataFrame(year_predictions)
            X_future = future_df[self.feature_names]
            
            # Масштабирование
            scale_features = [f for f in self.feature_names if not f.startswith(('lag', 'ОПЖ_MA', 'изменение_'))]
            X_future_scaled = X_future.copy()
            X_future_scaled[scale_features] = self.scaler.transform(X_future[scale_features])
            
            # Прогноз ОПЖ
            predictions = self.model.predict(X_future_scaled)
            
            # Сохраняем результаты в формате, понятном для сервера
            for i, (_, last_row) in enumerate(current_data.iterrows()):
                all_predictions.append({
                    'Регион': last_row['Регион'],
                    'Год': year,
                    'ОПЖ_прогноз': predictions[i],  # Переименовываем для единообразия
                    'ОПЖ_предыдущий': last_row['ОПЖ'],  # Фактическое значение за 2023
                    'Изменение_ОПЖ': predictions[i] - last_row['ОПЖ'],
                    'ОПЖ_базовый': last_row['ОПЖ'],
                    'Процент_изменения': ((predictions[i] - last_row['ОПЖ']) / last_row['ОПЖ']) * 100
                })
            
            # Обновляем последние данные для следующего года прогноза
            future_df['ОПЖ'] = predictions
            self.last_known_data = future_df.copy()
            
            # Статистика по году
            year_stats = all_predictions[-len(current_data):]
            avg_opj = np.mean([p['ОПЖ_прогноз'] for p in year_stats])
            avg_change = np.mean([p['Изменение_ОПЖ'] for p in year_stats])
            positive_changes = sum(1 for p in year_stats if p['Изменение_ОПЖ'] > 0)
            
            print(f"  Средняя ОПЖ: {avg_opj:.2f} лет")
            print(f"  Среднее изменение: {avg_change:+.3f} лет")
            print(f"  Регионов с ростом ОПЖ: {positive_changes}/{len(current_data)}")
        
        results_df = pd.DataFrame(all_predictions)
        
        # Анализ динамики
        self._analyze_predictions_dynamics(results_df, future_years)
        
        return results_df
    
    def _analyze_predictions_dynamics(self, predictions_df, years):
        """Анализ динамики прогнозов"""
        print(f"\n{'='*70}")
        print("АНАЛИЗ ДИНАМИКИ ПРОГНОЗОВ:")
        print(f"{'='*70}")
        
        for region in predictions_df['Регион'].unique()[:10]:  # Первые 10 регионов для примера
            region_data = predictions_df[predictions_df['Регион'] == region].sort_values('Год')
            if len(region_data) > 1:
                print(f"\n{region}:")
                for _, row in region_data.iterrows():
                    print(f"  {row['Год']}: {row['ОПЖ_прогноз']:.2f} лет "
                          f"(Δ: {row['Изменение_ОПЖ']:+.2f}, "
                          f"{row['Процент_изменения']:+.1f}%)")
        
        # Общая статистика
        print(f"\nОБЩАЯ СТАТИСТИКА ПО ГОДАМ:")
        for year in years:
            year_data = predictions_df[predictions_df['Год'] == year]
            avg_opj = year_data['ОПЖ_прогноз'].mean()
            max_opj = year_data['ОПЖ_прогноз'].max()
            min_opj = year_data['ОПЖ_прогноз'].min()
            
            print(f"\n{year} год:")
            print(f"  Средняя ОПЖ: {avg_opj:.2f} лет")
            print(f"  Максимальная: {max_opj:.2f} лет")
            print(f"  Минимальная: {min_opj:.2f} лет")
            print(f"  Размах: {max_opj - min_opj:.2f} лет")
    
    def save_model(self, filepath=None):
        """Сохранение модели"""
        if not self.is_fitted:
            print("Модель не обучена!")
            return False
        
        if filepath is None:
            filepath = os.path.join(output_dir, 'opj_xgboost_advanced_model.pkl')
        
        model_data = {
            'model': self.model,
            'scaler': self.scaler,
            'feature_names': self.feature_names,
            'first_year': self.first_year,
            'last_known_data': self.last_known_data,
            'validation_results': self.validation_results,
            'test_results': getattr(self, 'test_results', None),
            'random_state': self.random_state
        }
        
        joblib.dump(model_data, filepath)
        print(f"Модель ОПЖ сохранена в {filepath}")
        return True
    
    def load_model(self, filepath=None):
        """Загрузка модели"""
        if filepath is None:
            filepath = os.path.join(output_dir, 'opj_xgboost_advanced_model.pkl')
            
        if not os.path.exists(filepath):
            print(f"Файл модели {filepath} не найден!")
            return False
        
        model_data = joblib.load(filepath)
        self.model = model_data['model']
        self.scaler = model_data['scaler']
        self.feature_names = model_data['feature_names']
        self.first_year = model_data['first_year']
        self.last_known_data = model_data['last_known_data']
        self.validation_results = model_data['validation_results']
        self.test_results = model_data['test_results']
        self.random_state = model_data.get('random_state', 42)
        self.is_fitted = True
        
        print(f"Модель ОПЖ загружена из {filepath}")
        return True
    
    def prepare_final_output_for_server(self, predictions_df, target_year=2024):
        """
        Подготовка финального файла в формате для сервера
        Только 4 поля: Регион, Год, ОПЖ, predictions
        """
        # Фильтруем данные за нужный год
        year_data = predictions_df[predictions_df['Год'] == target_year]
        
        final_output = pd.DataFrame({
            'Регион': year_data['Регион'],
            'Год': year_data['Год'],
            'ОПЖ': year_data['ОПЖ_предыдущий'],  # Фактическое значение
            'predictions': year_data['ОПЖ_прогноз']  # Прогноз
        })
        
        return final_output

# Главная функция для ОПЖ
def run_opj_pipeline():
    """
    Запуск пайплайна для ОПЖ с созданием predictions_ele.xlsx
    """
    print("="*80)
    print("ПАЙПЛАЙН ДЛЯ ОПЖ (predictions_ele.xlsx)")
    print("="*80)
    
    # Создаем директорию для результатов
    os.makedirs(output_dir, exist_ok=True)
    
    # Загрузка данных
    print("Загрузка данных ОПЖ...")
    try:
        df_opj = pd.read_excel('Финальный вариант/общая_ОПЖ (2).xlsx')
    except FileNotFoundError:
        print("ОШИБКА: Файл 'общая_ОПЖ (2).xlsx' не найден!")
        print("Убедитесь, что файл находится в папке 'Финальный вариант'")
        return None
    
    # Стандартизация регионов
    print("Стандартизация названий регионов...")
    df_opj = standardize_region_names(df_opj)
    print(f"Уникальных регионов: {df_opj['Регион'].nunique()}")
    print(f"Период данных: {df_opj['Год'].min()}-{df_opj['Год'].max()}")
    
    print(f"\n{'='*70}")
    print("АЛЬТЕРНАТИВНЫЙ ПОДХОД ДЛЯ БОЛЕЕ ТОЧНОГО ПРОГНОЗА ОПЖ")
    print(f"{'='*70}")
    
    # Создание и обучение модели по альтернативному подходу
    opj_forecaster = OPJXGBoostForecasterAdvanced()
    
    # Шаг 1: Обучаем модель на данных до 2022 года, валидируем на 2022 году
    opj_forecaster.fit_with_validation(df_opj, train_end_year=2021, validation_year=2022)
    
    # Шаг 2: Тестируем модель на реальных данных 2023 года
    test_metrics = opj_forecaster.test_on_2023(df_opj)
    
    # Сохранение модели после тестирования
    opj_forecaster.save_model()
    
    # Шаг 3: Прогноз на 2024 и 2025 годы
    print(f"\n{'='*70}")
    print("СОЗДАНИЕ ПРОГНОЗА ОПЖ НА 2024-2025 ГОДЫ")
    print(f"{'='*70}")
    
    future_predictions = opj_forecaster.predict_future_advanced([2024, 2025])
    
    if future_predictions is not None:
        # ГАРАНТИРОВАННОЕ СОЗДАНИЕ ФАЙЛА predictions_ele.xlsx для 2024 года
        print(f"\n{'='*70}")
        print("СОЗДАНИЕ ФАЙЛА ДЛЯ СЕРВЕРА: predictions_ele.xlsx")
        print(f"{'='*70}")
        
        # Подготовка данных для 2024 года в формате для сервера
        server_data_2024 = opj_forecaster.prepare_final_output_for_server(future_predictions, target_year=2024)
        
        # Путь к файлу для сервера
        predictions_filepath = os.path.join(output_dir, 'predictions_ele.xlsx')
        
        # Сохраняем файл
        server_data_2024.to_excel(predictions_filepath, index=False)
        print(f"Файл для сервера сохранен: {predictions_filepath}")
        print(f"Размер файла: {server_data_2024.shape[0]} строк, {server_data_2024.shape[1]} колонок")
        print(f"Год прогноза: {server_data_2024['Год'].iloc[0]}")
        print(f"Уникальных регионов: {server_data_2024['Регион'].nunique()}")
        
        # Проверка структуры файла
        print("\nСТРУКТУРА ФАЙЛА predictions_ele.xlsx:")
        print(f"Колонки: {list(server_data_2024.columns)}")
        print(f"Типы данных: {server_data_2024.dtypes.to_dict()}")
        
        # Пример данных
        print("\nПЕРВЫЕ 5 СТРОК ФАЙЛА:")
        print(server_data_2024.head())
        
        # Статистика
        print("\nСТАТИСТИКА ПРОГНОЗА ОПЖ НА 2024:")
        print(f"Среднее ОПЖ (факт 2023): {server_data_2024['ОПЖ'].mean():.2f} лет")
        print(f"Средний прогноз ОПЖ (2024): {server_data_2024['predictions'].mean():.2f} лет")
        print(f"Средний рост: {server_data_2024['predictions'].mean() - server_data_2024['ОПЖ'].mean():+.2f} лет")
        
        # Также создаем файл для 2025 года (дополнительно)
        server_data_2025 = opj_forecaster.prepare_final_output_for_server(future_predictions, target_year=2025)
        predictions_2025_filepath = os.path.join(output_dir, 'predictions_ele_2025.xlsx')
        server_data_2025.to_excel(predictions_2025_filepath, index=False)
        print(f"\nДополнительный файл для 2025 года: {predictions_2025_filepath}")
        
        # Сохраняем полные результаты для анализа
        full_results_filepath = os.path.join(output_dir, 'full_opj_predictions_2024_2025_advanced.csv')
        future_predictions.to_csv(full_results_filepath, index=False, encoding='utf-8-sig')
        print(f"Полные результаты сохранены: {full_results_filepath}")
        
        print(f"\n{'='*70}")
        print("ПАЙПЛАЙН ОПЖ УСПЕШНО ЗАВЕРШЕН!")
        print("Файл predictions_ele.xlsx готов для загрузки в Git")
        print(f"{'='*70}")
        
        return predictions_filepath
    
    return None

if __name__ == "__main__":
    # Запускаем пайплайн для ОПЖ
    predictions_file = run_opj_pipeline()
    
    if predictions_file:
        print(f"\nПайплайн выполнен успешно!")
        print(f"Файл для сервера создан: {predictions_file}")
    else:
        print("\nПайплайн завершился с ошибкой")